# Code example - running a profit simulation

#### Maria Silva, September 2024

## Import packages

In [1]:
import os

In [2]:
import sys
code_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "src"))

sys.path.append(code_dir)

In [3]:
from rollup import RollupSpec
from gas import GasPriceModel
from asset import AssetPriceModel
from extraction import run_arb_profit_simulation

## Define simulation paramaters

In [4]:
# Number of iterations (i.e. Monte-carlo samples)
n_iter = 10

In [5]:
# Define rollup settings
arb_pool_fee = 0.005 # The arb pool fee must be the same in both rollups!
price_A = 105.0
price_B = 100.0 # price_A must be higher than price_B!
arb_pool_reserve_x_A = 1000.0
arb_pool_reserve_x_B = 1000.0

In [6]:
# Init spec classes - rollup A
gas_price_model_A = GasPriceModel(
    model_type="gaussian", gas_price_mean=0.01, gas_price_std=0.0001
)
rollup_A = RollupSpec(
    fail_rate=0.5,
    gas_price_model=gas_price_model_A,
    gas_units_swap=10.0,
    gas_units_fail=1.0,
    arb_pool_reserve_x=arb_pool_reserve_x_A,
    arb_pool_reserve_y=arb_pool_reserve_x_A*price_A, 
    arb_pool_fee=arb_pool_fee,
)

# Init spec classes  - rollup B
gas_price_model_B = GasPriceModel(
    model_type="gaussian", gas_price_mean=0.01, gas_price_std=0.0001
)
rollup_B = RollupSpec(
    fail_rate=0.5,
    gas_price_model=gas_price_model_A,
    gas_units_swap=10.0,
    gas_units_fail=1.0,
    arb_pool_reserve_x=arb_pool_reserve_x_B,
    arb_pool_reserve_y=arb_pool_reserve_x_B*price_B,
    arb_pool_fee=arb_pool_fee,
)

In [7]:
# Define asset price settings
y_price_model = AssetPriceModel(
    asset_label="X", fee=0.005, model_type="constant", asset_price_mean=50.0
)

## Run simulation

In [8]:
# Run simulation

arb_sim_df = run_arb_profit_simulation(n_iter, rollup_A, rollup_B, y_price_model)
arb_sim_df

,iter,fail_outcome_A,fail_outcome_B,gas_price_A,gas_price_B,pure_bundle_profit_A,pure_bundle_profit_B,atomic_bundle_profit,non_atomic_bundle_profit,atomic_arb_cost,non_atomic_arb_cost,atomic_profit,non_atomic_profit,shared_sequencing_gain
0,0,1,1,0.010023,0.010118,-0.000000,-0.000000,0.000000,-0.000000,0.020142,0.020142,-0.020142,-0.020142,0.000000
1,1,0,0,0.010100,0.009894,-1083.238798,-1062.445047,-2145.683845,-2145.683845,0.199942,0.199942,-2145.883786,-2145.883786,0.000000
2,2,0,0,0.010069,0.010112,-1083.238798,-1062.445047,-2145.683845,-2145.683845,0.201810,0.201810,-2145.885654,-2145.885654,0.000000
3,3,1,0,0.010022,0.009824,-0.000000,-1062.445047,0.000000,-1062.445047,0.019846,0.108264,-0.019846,-1062.553311,1062.533465
4,4,0,1,0.010010,0.010044,-1083.238798,-0.000000,0.000000,-1083.238798,0.020054,0.110145,-0.020054,-1083.348943,1083.328889
5,5,0,0,0.009860,0.010023,-1083.238798,-1062.445047,-2145.683845,-2145.683845,0.198837,0.198837,-2145.882682,-2145.882682,0.000000
6,6,0,1,0.009941,0.009972,-1083.238798,-0.000000,0.000000,-1083.238798,0.019913,0.109377,-0.019913,-1083.348175,1083.328262
7,7,1,1,0.010057,0.010264,-0.000000,-0.000000,0.000000,-0.000000,0.020322,0.020322,-0.020322,-0.020322,0.000000
8,8,1,1,0.009940,0.010082,-0.000000,-0.000000,0.000000,-0.000000,0.020022,0.020022,-0.020022,-0.020022,0.000000
9,9,1,0,0.009972,0.010145,-0.000000,-1062.445047,0.000000,-1062.445047,0.020117,0.111420,-0.020117,-1062.556467,1062.536350


In [9]:
arb_sim_df["shared_sequencing_gain"].agg(["mean", "std"])

mean    429.172697
std     554.102922
Name: shared_sequencing_gain, dtype: float64